<a href="https://colab.research.google.com/github/principioincertidumbre/ComputerVision/blob/main/IntroCV_Tarea_4_Sof%C3%ADa_Vits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <h1>Introducción a la Visión Computacional</h1>
    <h1>Tarea 4</h1>
</center>

### Magíster en Data Science
### U. del Desarrollo

-----------------------
<div align="right">
Fecha de Entrega: Domingo 25, Mayo 2025.
</div>

1. Proponer mejoras al sistema diseñado e implementado originalmete.

2. Volver a investigar, aplicar y/o ajustar un algoritmo diferente al algoritmo utilizado trabajado en la Tarea 2, o bien una extensión de éste.

3. Implementar métricas de comparación, por ejemplo precisión o una matriz de confusión, junto con otros indicadores tales como tiempos de ejecución de el procesamiento del dataset. La comparación debiera ser con el algoritmo de la Tarea 2. Si resultó que no fue satisfactorio en esta Tarea 2, implementar más de 1 algoritmo o procedimiento para esta Tarea, de manera de poder realizar la comparación mencionada.

4. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) de la comparación con los resultados de la Tarea 2. Notar que pudiera ser que el resultado es inferior al de la Tarea 2, lo importante es el comentario o discusión del por qué se mejoró o empeoraron los nuevos resultados.

Tan o más importante que los resultados propiamente tales es la discusión y comentarios acerca de la implementación y los resultados. Como siempre, apoyarse en tablas, gráficos e imágenes.


# Introducción

La clasificación de imágenes es un método utilizado para categorizar automáticamente datos visuales según sus características. Por ejemplo, permite diferenciar entre fotografías de tejidos sanos y enfermos de distintos seres vivos. En el caso de este trabajo esta metodología permite distinguir entre papas sanas e infectadas por bacterias y hongos, lo cual tiene una importancia económica y de seguridad alimentaria debido a la relevancia del cultivo de la papa para la alimentación humana.

Previamente se pre-procesó el dataset "Potato Diseases Datasets" aplicando desenfoque Gaussiano, detección de bordes mediante la aplicación del filtro Sobel, aclarado de las imágenes mediante filtro Máximo, y oscurecimiento de imágenes al aplicar el filtro Mínimo.

Este dataset contiene 451 imágenes de papas clasificadas como infectadas por common scab, blackleg, dry rot, pink rot, y black scurf, papas sanas, y papas afectadas por otros tipos de infecciones.

Considerando las enfermedades más comunes de la papa en Chile, se consideraron las clases common scab, blackleg, dry rot y papas sanas para realizar una clasificación multiclase mediante la aplicación del algoritmo . Esta muestra del dataset contiene 262 imágenes en total.

Finalmente, se determinó el algoritmo más adecuado para la clasificación de la muestra del dataset de papas.

Fuente del dataset utilizado:

https://www.kaggle.com/datasets/mukaffimoin/potato-diseases-datasets/

F. T. J. Faria, M. Bin Moin, A. Al Wase, M. R. Sani, K. M. Hasib and M. S. Alam, "Classification of Potato Disease with Digital Image Processing Technique: A Hybrid Deep Learning Framework," 2023 IEEE 13th Annual Computing and Communication Workshop and Conference (CCWC), Las Vegas, NV, USA, 2023, pp. 0820-0826, doi: 10.1109/CCWC57344.2023.10099162.

# Metodología

1.- Cargar la muestra del dataset y ordenar las carpetas a utilizar para el algoritmo de clasificación  multiclase.

2.- Implementar un algoritmo de clasificación, midiendo el tiempo mínimo, tiempo máximo, tiempo promedio y desviación estándard del tiempo de procesamiento.

3.- Comparar los resultados del algoritmo con los resultados obtenidos al utilizar el algoritmo SVM mediante métricas para verificar qué tipo de preprocesamiento permite obtener los mejores resultados.


# Desarrollo

## Carga de librerías y definición de rutas de las carpetas que contienen las imágenes de la muestra del dataset de papas

In [ ]:
from PIL import Image
from PIL import ImageDraw
from os.path import exists
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import pandas as pd
import time
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Abrir una imagen de cada carpeta que contiene imágenes de papas enfermas o sanas

inPath1  = '/content/drive/MyDrive/potato/data/common_scab'
inPath2  = '/content/drive/MyDrive/potato/data/blackleg'
inPath3  = '/content/drive/MyDrive/potato/data/dry_rot'
inPath4 = '/content/drive/MyDrive/potato/data/healthy_potato'
inPath5 = '/content/drive/MyDrive/potato/data_g/common_scab'
inPath6 = '/content/drive/MyDrive/potato/data_g/blackleg'
inPath7 = '/content/drive/MyDrive/potato/data_g/dry_rot'
inPath8 = '/content/drive/MyDrive/potato/data_g/healthy_potato'
inPath9 = '/content/drive/MyDrive/potato/data_s/common_scab'
inPath10 = '/content/drive/MyDrive/potato/data_s/blackleg'
inPath11 = '/content/drive/MyDrive/potato/data_s/dry_rot'
inPath12 = '/content/drive/MyDrive/potato/data_s/healthy_potato'
inPath13 = '/content/drive/MyDrive/potato/data_max/common_scab'
inPath14 = '/content/drive/MyDrive/potato/data_max/blackleg'
inPath15 = '/content/drive/MyDrive/potato/data_max/dry_rot'
inPath16 = '/content/drive/MyDrive/potato/data_max/healthy_potato'
inPath17 = '/content/drive/MyDrive/potato/data_min/common_scab'
inPath18 = '/content/drive/MyDrive/potato/data_min/blackleg'
inPath19 = '/content/drive/MyDrive/potato/data_min/dry_rot'
inPath20 = '/content/drive/MyDrive/potato/data_min/healthy_potato'

## Resultado del pre-procesamiento del dataset de papas

En esta sección se muestran ejemplos de papas sanas e infectadas por blackleg, common scab, y dry rot. En todos los casos se comparan imágenes sin filtrar con fotografías a las cuales se les aplicó desenfoque Gaussiano, detección de bordes mediante un filtro Sobel, y filtros Máximo y Mínimo.

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size


    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
# Compara papa afectada por blackleg antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im = Image.open('{}/5.jpg'.format(inPath2))

from PIL import ImageDraw
ImageDraw.Draw(pil_im).text((0, 0),'Blackleg', (0, 0, 0))

display(pil_im)

# Desenfoque Gaussiano
pil_im_g = Image.open('{}/5.jpg'.format(inPath6)).convert('RGB')
ImageDraw.Draw(pil_im_g).text((0, 0),'Blackleg Gaussian', (0, 0, 0))

# Sobel
pil_im_s = Image.open('{}/5.jpg'.format(inPath10)).convert('RGB')
ImageDraw.Draw(pil_im_s).text((0, 0),'Blackleg Sobel', (255, 255, 255))

# Máximo
pil_im_m = Image.open('{}/5.jpg'.format(inPath14)).convert('RGB')
ImageDraw.Draw(pil_im_m).text((0, 0),'Blackleg Maximum', (0, 0, 0))

# Mínimo
pil_im_mi = Image.open('{}/5.jpg'.format(inPath18)).convert('RGB')
ImageDraw.Draw(pil_im_mi).text((0, 0),'Blackleg Minimum', (0, 0, 0))

imgs = [pil_im_g, pil_im_s, pil_im_m, pil_im_mi]

grid = image_grid(imgs, rows=2, cols=2,)

display(grid)

In [ ]:
# Compara papa afectada por blackleg antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im2 = Image.open('{}/50.jpg'.format(inPath2))
ImageDraw.Draw(pil_im2).text((0, 0),'Blackleg', (255, 255, 255))
display(pil_im2)

# Desenfoque Gaussiano
pil_im_g2 = Image.open('{}/50.jpg'.format(inPath6)).convert('RGB')
ImageDraw.Draw(pil_im_g2).text((0, 0),'Blackleg Gaussian', (255, 255, 255))

# Sobel
pil_im_s2 = Image.open('{}/50.jpg'.format(inPath10)).convert('RGB')
ImageDraw.Draw(pil_im_s2).text((0, 0),'Blackleg Sobel', (255, 255, 255))

# Máximo
pil_im_m2 = Image.open('{}/50.jpg'.format(inPath14)).convert('RGB')
ImageDraw.Draw(pil_im_m2).text((0, 0),'Blackleg Maximum', (255, 255, 255))

# Mínimo
pil_im_mi2 = Image.open('{}/50.jpg'.format(inPath18)).convert('RGB')
ImageDraw.Draw(pil_im_mi2).text((0, 0),'Blackleg Minimum', (255, 255, 255))

imgs = [pil_im_g2, pil_im_s2, pil_im_m2, pil_im_mi2]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

In [ ]:
# Compara papa afectada por common scab antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im3 = Image.open('{}/1.jpg'.format(inPath1))
ImageDraw.Draw(pil_im3).text((0, 0),'Common scab', (0, 0, 0))
display(pil_im3)

# Desenfoque Gaussiano
pil_im_g3 = Image.open('{}/1.jpg'.format(inPath5)).convert('RGB')
ImageDraw.Draw(pil_im_g3).text((0, 0),'C. scab Gaussian', (0, 0, 0))

# Sobel
pil_im_s3 = Image.open('{}/1.jpg'.format(inPath9)).convert('RGB')
ImageDraw.Draw(pil_im_s3).text((0, 0),'C. scab Sobel', (255, 255, 255))

# Máximo
pil_im_m3 = Image.open('{}/1.jpg'.format(inPath13)).convert('RGB')
ImageDraw.Draw(pil_im_m3).text((0, 0),'C. scab Maximum', (0, 0, 0))

# Mínimo
pil_im_mi3 = Image.open('{}/1.jpg'.format(inPath17)).convert('RGB')
ImageDraw.Draw(pil_im_mi3).text((0, 0),'C. scab Minimum', (0, 0, 0))

imgs = [pil_im_g3, pil_im_s3, pil_im_m3, pil_im_mi3]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

In [ ]:
# Compara papa afectada por common scab antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im4 = Image.open('{}/30.jpg'.format(inPath1))
ImageDraw.Draw(pil_im4).text((0, 0),'Common scab', (0, 0, 0))
display(pil_im4)

# Desenfoque Gaussiano
pil_im_g4 = Image.open('{}/30.jpg'.format(inPath5)).convert('RGB')
ImageDraw.Draw(pil_im_g4).text((0, 0),'C. scab Gaussian', (0, 0, 0))

# Sobel
pil_im_s4 = Image.open('{}/30.jpg'.format(inPath9)).convert('RGB')
ImageDraw.Draw(pil_im_s4).text((0, 0),'C. scab Sobel', (255, 255, 255))

# Máximo
pil_im_m4 = Image.open('{}/30.jpg'.format(inPath13)).convert('RGB')
ImageDraw.Draw(pil_im_m4).text((0, 0),'C. scab Maximum', (0, 0, 0))

# Mínimo
pil_im_mi4 = Image.open('{}/30.jpg'.format(inPath17)).convert('RGB')
ImageDraw.Draw(pil_im_mi4).text((0, 0),'C. scab Minimum', (0, 0, 0))

imgs = [pil_im_g4, pil_im_s4, pil_im_m4, pil_im_mi4]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

In [ ]:
# Compara papa afectada por dry rot antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im5 = Image.open('{}/31.jpg'.format(inPath3))
ImageDraw.Draw(pil_im5).text((0, 0),'Dry rot', (255, 255, 255))
display(pil_im5)

# Desenfoque Gaussiano
pil_im_g5 = Image.open('{}/31.jpg'.format(inPath7)).convert('RGB')
ImageDraw.Draw(pil_im_g5).text((0, 0),'D. rot Gaussian', (255, 255, 255))

# Sobel
pil_im_s5 = Image.open('{}/31.jpg'.format(inPath11)).convert('RGB')
ImageDraw.Draw(pil_im_s5).text((0, 0),'D. rot Sobel', (255, 255, 255))

# Máximo
pil_im_m5 = Image.open('{}/31.jpg'.format(inPath15)).convert('RGB')
ImageDraw.Draw(pil_im_m5).text((0, 0),'D. rot Maximum', (255, 255, 255))

# Mínimo
pil_im_mi5 = Image.open('{}/31.jpg'.format(inPath19)).convert('RGB')
ImageDraw.Draw(pil_im_mi5).text((0, 0),'D. rot Minimum', (255, 255, 255))

imgs = [pil_im_g5, pil_im_s5, pil_im_m5, pil_im_mi5]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

In [ ]:
# Compara papa afectada por dry rot antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im6 = Image.open('{}/55.jpg'.format(inPath3)).convert('RGB')
ImageDraw.Draw(pil_im6).text((0, 0),'Dry rot', (0, 0, 0))
display(pil_im6)

# Desenfoque Gaussiano
pil_im_g6 = Image.open('{}/55.jpg'.format(inPath7)).convert('RGB')
ImageDraw.Draw(pil_im_g6).text((0, 0),'D. rot Gaussian', (0, 0, 0))

# Sobel
pil_im_s6 = Image.open('{}/55.jpg'.format(inPath11)).convert('RGB')
ImageDraw.Draw(pil_im_s6).text((0, 0),'D. rot Sobel', (255, 255, 255))

# Máximo
pil_im_m6 = Image.open('{}/55.jpg'.format(inPath15)).convert('RGB')
ImageDraw.Draw(pil_im_m6).text((0, 0),'D. rot Maximum', (0, 0, 0))

# Mínimo
pil_im_mi6 = Image.open('{}/55.jpg'.format(inPath19)).convert('RGB')
ImageDraw.Draw(pil_im_mi6).text((0, 0),'D. rot Minimum', (0, 0, 0))

imgs = [pil_im_g6, pil_im_s6, pil_im_m6, pil_im_mi6]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

In [ ]:
# Compara papa sana antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im7 = Image.open('{}/55.jpg'.format(inPath4))
ImageDraw.Draw(pil_im7).text((0, 0),'Healthy', (0, 0, 0))
display(pil_im7)

# Desenfoque Gaussiano
pil_im_g7 = Image.open('{}/55.jpg'.format(inPath8)).convert('RGB')
ImageDraw.Draw(pil_im_g7).text((0, 0),'Healthy Gaussian', (0, 0, 0))

# Sobel
pil_im_s7 = Image.open('{}/55.jpg'.format(inPath12)).convert('RGB')
ImageDraw.Draw(pil_im_s7).text((0, 0),'Healthy Sobel', (255, 255, 255))

# Máximo
pil_im_m7 = Image.open('{}/55.jpg'.format(inPath16)).convert('RGB')
ImageDraw.Draw(pil_im_m7).text((0, 0),'Healthy Maximum', (0, 0, 0))

# Mínimo
pil_im_mi7 = Image.open('{}/55.jpg'.format(inPath20)).convert('RGB')
ImageDraw.Draw(pil_im_mi7).text((0, 0),'Healthy Minimum', (0, 0, 0))

imgs = [pil_im_g7, pil_im_s7, pil_im_m7, pil_im_mi7]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

In [ ]:
# Compara papa sana antes y después de aplicar desenfoque Gaussiano, filtros Sobel, Máximo y Mínimo

# Original
pil_im8 = Image.open('{}/5.jpg'.format(inPath4))
ImageDraw.Draw(pil_im8).text((0, 0),'Healthy', (0, 0, 0))
display(pil_im8)

# Desenfoque Gaussiano
pil_im_g8 = Image.open('{}/5.jpg'.format(inPath8)).convert('RGB')
ImageDraw.Draw(pil_im_g8).text((0, 0),'Healthy Gaussian', (0, 0, 0))

# Sobel
pil_im_s8 = Image.open('{}/5.jpg'.format(inPath12)).convert('RGB')
ImageDraw.Draw(pil_im_s8).text((0, 0),'Healthy Sobel', (255, 255, 255))

# Máximo
pil_im_m8 = Image.open('{}/5.jpg'.format(inPath16)).convert('RGB')
ImageDraw.Draw(pil_im_m8).text((0, 0),'Healthy Maximum', (0, 0, 0))

# Mínimo
pil_im_mi8 = Image.open('{}/5.jpg'.format(inPath20)).convert('RGB')
ImageDraw.Draw(pil_im_mi8).text((0, 0),'Healthy Minimum', (0, 0, 0))

imgs = [pil_im_g8, pil_im_s8, pil_im_m8, pil_im_mi8]

grid = image_grid(imgs, rows=2, cols=2)

display(grid)

## Clasificador  para dataset sin filtrar

In [ ]:
ex_time_nf = []
half_time_nf = []

j = 1

while j < 16:
    start = time.perf_counter()

end = time.perf_counter()
    total = end - start  # calcula tiempo total
    print(f"Tiempo transcurrido: {(total)} s")
    ex_time_nf.append(total)
    half_t = (total/2)  # calcula valor para aproximar el tiempo que toma procesar el 50% de las imágenes
    half_time_nf.append(half_t)
    j +=1

## Determinación del tiempo de ejecución para el 50% y 100% de las imágenes clasificadas   

In [ ]:
print("Tiempo mínimo ", min(ex_time_nf), 's')
print("Tiempo máximo: ", max(ex_time_nf), 's')
print("Tiempo promedio: ", sum(ex_time_nf)/len(ex_time_nf), 's')
print("Desviación estándar del tiempo: ", np.std(ex_time_nf), 's')

In [ ]:
print("Tiempo mínimo mitad ejecución ", min(half_time_nf), 's')
print("Tiempo máximo mitad ejecución: ", max(half_time_nf), 's')
print("Tiempo promedio mitad ejecución: ", sum(half_time_nf)/len(half_time_nf), 's')
print("Desviación estándar del tiempo mitad ejecución: ", np.std(half_time_nf), 's')

## Clasificador  para dataset filtro Gaussiano

In [ ]:
ex_time = []
half_time = []

j = 1

while j < 16:
    start = time.perf_counter()

end = time.perf_counter()
    total = end - start
    print(f"Tiempo transcurrido: {(total)} s")
    ex_time.append(total)
    half_t = (total/2)
    half_time.append(half_t)
    j +=1

## Determinación del tiempo de ejecución para el 50% y 100% de las imágenes clasificadas

In [ ]:
print("Tiempo mínimo ", min(ex_time), 's')
print("Tiempo máximo: ", max(ex_time), 's')
print("Tiempo promedio: ", sum(ex_time)/len(ex_time),'s')
print("Desviación estándar del tiempo: ", np.std(ex_time), 's')

In [ ]:
print("Tiempo mínimo mitad ejecución ", min(half_time), 's')
print("Tiempo máximo mitad ejecución: ", max(half_time), 's')
print("Tiempo promedio mitad ejecución: ", sum(half_time)/len(half_time), 's')
print("Desviación estándar del tiempo mitad ejecución: ", np.std(half_time), 's')

## Clasificador  para dataset filtro Sobel

In [ ]:
ex_time_s = []
half_time_s = []

j = 1

while j < 16:
    start = time.perf_counter()

end = time.perf_counter()
    total = end - start
    print(f"Tiempo transcurrido: {(total)} s")
    ex_time_s.append(total)
    half_t = (total/2)
    half_time_s.append(half_t)
    j +=1


## Determinación del tiempo de ejecución para el 50% y 100% de las imágenes clasificadas

In [ ]:
print("Tiempo mínimo ", min(ex_time_s), 's')
print("Tiempo máximo: ", max(ex_time_s), 's')
print("Tiempo promedio: ", sum(ex_time_nf)/len(ex_time_s), 's')
print("Desviación estándar del tiempo: ", np.std(ex_time_s), 's')

In [ ]:
print("Tiempo mínimo mitad ejecución ", min(half_time_s), 's')
print("Tiempo máximo mitad ejecución: ", max(half_time_s), 's')
print("Tiempo promedio mitad ejecución: ", sum(half_time_s)/len(half_time_s), 's')
print("Desviación estándar del tiempo mitad ejecución: ", np.std(half_time_s), 's')

## Clasificador  para dataset filtro Máximo

In [ ]:
ex_time_max = []
half_time_max = []

j = 1

while j < 16:
    start = time.perf_counter()


  end = time.perf_counter()
    total = end - start
    print(f"Tiempo transcurrido: {(total)} s")
    ex_time_max.append(total)
    half_t = (total/2)
    half_time_max.append(half_t)
    j +=1


## Determinación del tiempo de ejecución para el 50% y 100% de las imágenes clasificadas

In [ ]:
print("Tiempo mínimo ", min(ex_time_max), 's')
print("Tiempo máximo: ", max(ex_time_max), 's')
print("Tiempo promedio: ", sum(ex_time_max)/len(ex_time_max), 's')
print("Desviación estándar del tiempo: ", np.std(ex_time_max), 's')

In [ ]:
print("Tiempo mínimo mitad ejecución ", min(half_time_max), 's')
print("Tiempo máximo mitad ejecución: ", max(half_time_max))
print("Tiempo promedio mitad ejecución: ", sum(half_time_max)/len(half_time_max), 's')
print("Desviación estándar del tiempo mitad ejecución: ", np.std(half_time_max), 's')

## Clasificador  para dataset filtro Mínimo

In [ ]:
ex_time_min = []
half_time_min = []

j = 1

while j < 16:
    start = time.perf_counter()

 end = time.perf_counter()
    total = end - start
    print(f"Tiempo transcurrido: {(total)} s")
    ex_time_min.append(total)
    half_t = (total/2)
    half_time_min.append(half_t)
    j +=1


## Determinación del tiempo de ejecución para el 50% y 100% de las imágenes clasificadas

In [ ]:
print("Tiempo mínimo ", min(ex_time_min), 's')
print("Tiempo máximo: ", max(ex_time_min), 's')
print("Tiempo promedio: ", sum(ex_time_min)/len(ex_time_min), 's')
print("Desviación estándar del tiempo: ", np.std(ex_time_min), 's')

In [ ]:
print("Tiempo mínimo mitad ejecución ", min(half_time_min), 's')
print("Tiempo máximo mitad ejecución: ", max(half_time_min), 's')
print("Tiempo promedio mitad ejecución: ", sum(half_time_min)/len(half_time_min), 's')
print("Desviación estándar del tiempo mitad ejecución: ", np.std(half_time_min), 's')

## Análisis de resultados del procesamiento de la muestra del dataset de papas

|**Modelo**| **Accuracy**| **Precision** | **Recall** | **F1 Score**|
|---|---|---|---|---|
| **Modelo 1 (SVM sin filtro)** |64.15 %| 63.41 %| 64.15 %| 62.58 %|,
| **Modelo 2 (SVM filtro Gaussiano)** | 56.60 % | 61.22 %| 56.60 %| 57.74 %|
| **Modelo 3 (SVM filtro Sobel)** | 81.13 % | 81.85 %| 81.13 %| 80.96 % |
|**Modelo 4 (SVM filtro Máximo)** | 60.38 % | 59.30 %| 60.38 %| 59.35 % |
| **Modelo 5 (SVM filtro Mínimo)** |56.60 % | 66.51 %| 56.60 %| 57.54 % |

 | **Modelo**      | **t min (s)**| **t máx (s)** | **t prom. (s)** | **std. t (s)**|
|---|---|---|---|---|
| **Modelo 1 (SVM sin filtro)** |14.01| 22.52| 15.71| 2.18|
| **Modelo 2 (SVM filtro Gaussiano)** | 14.39| 18.24| 15.72| 1.35|
| **Modelo 3 (SVM filtro Sobel)** | 14.13| 29.14|15.71| 5.34|
| **Modelo 4 (SVM filtro Máximo)** |14.74| 34.60| 22.04|6.15|
| **Modelo 5 (SVM filtro Mínimo)** |14.87|30.03|21.65|5.26|

|**Modelo**| **t/2 min (s)**| **t/2 máx (s)** | **t/2 prom. (s)** | **std. t/2 (s)**|
|---|---|---|---|---|        
|**Modelo 1 (SVM sin filtro)** |7.01| 11.26| 7.85|  1.09|
|**Modelo 2 (SVM filtro Gaussiano)** | 7.19| 9.12|7.86|0.68|
|**Modelo 3 (SVM filtro Sobel)** |7.06|14.57|9.55|2.67|
| **Modelo 4 (SVM filtro Máximo)** |7.37|17.30| 10.83|3.08|
| **Modelo 5 (SVM filtro Mínimo)** |7.44| 15.01|8.54|2.63|

# Conclusiones